In [1]:
#Hybrid Recommender

In [7]:
#Common libraries
import pandas as pd
import numpy as np
from surprise import SVD, Reader, Dataset

In [31]:
#Uploading the datasets
cosine_sim = pd.read_csv('cosine_sim.csv')

cosine_sim_map = pd.read_csv('cosine_sim_map.csv', header=None)
cosine_sim_map = cosine_sim_map.set_index(0)
cosine_sim_map = cosine_sim_map[1]

In [32]:
from surprise import SVD, Reader, Dataset

reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)
svd = SVD()
train_set = data.build_full_trainset()
svd.fit(train_set)

In [35]:
id_map = pd.read_csv('movie_ids.csv')
id_to_title = id_map.set_index('id')
title_to_id = id_map.set_index('title')

In [36]:
smd = pd.read_csv('metadata_small.csv')


In [37]:
#Function for hybrid recommender
def hybrid(user_id, title):
    idx = cosine_sim_map[title]    
    sim_scores = list(enumerate(cosine_sim[str(int(idx))]))    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)    
    sim_scores = sim_scores[1:26]    
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]    
    movies['est'] = movies['id'].apply(lambda x: svd.predict(user_id, id_to_title.loc[x]['movieId']).est)    
    movies = movies.sort_values('est', ascending=False)    
    return movies.head(10)

In [40]:
#First Attempt
hybrid(2, 'Inception')

,title,vote_count,vote_average,year,id,est
7502,The Book of Eli,2207.0,6.6,2010,20504,3.671466
7901,Super 8,2496.0,6.6,2011,37686,3.666224
6640,Déjà Vu,1519.0,6.6,2006,7551,3.650746
4173,Minority Report,2663.0,7.1,2002,180,3.595827
7948,Stake Land,290.0,6.2,2010,52015,3.544085
5580,The Three Lives of Thomasina,12.0,6.8,1963,15081,3.543062
477,The Shadow,140.0,5.4,1994,8850,3.537813
5970,Cube²: Hypercube,383.0,5.4,2002,437,3.530128
7208,Replicant,93.0,5.0,2001,10596,3.464943
5296,Zardoz,106.0,5.8,1974,4923,3.453331
